In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys


import functions.parse_data as parse
from functions.parse_data import synth_dataloader
import functions.handy_functions as hf
import torch.nn as nn
import torch

from multivariate_quantile_regression.network_model import QuantileNetwork

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm

import os
import random

from cot_train.utils import MLP5

/tmp/ipykernel_151915/4205888445.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-21 13:41:41.832926: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 13:41:41.866680: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 13:4

In [3]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)
device = tensor.device

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [3]:
#Find same trainval/test split as Aleksis alg.
#Load training, val and test data for indices
traindata = pd.read_csv('cot_train/data/synthetic-cot-data/train_df.csv', index_col=[0])
valdata = pd.read_csv('cot_train/data/synthetic-cot-data/val_df.csv', index_col=[0])
testdata = pd.read_csv('cot_train/data/synthetic-cot-data/test_df.csv', index_col=[0])

#Merge train and val data
trainvaldata = pd.concat([traindata,valdata])

#Split X's and y's
X_cols = ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
          'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12']
y_cols = ['COT']

X_trainval = trainvaldata[X_cols]
X_test = testdata[X_cols]
y_trainval = trainvaldata[y_cols]
y_test = testdata[y_cols]

#Add noise to X_test
np.random.seed(313)
X_test = X_test + np.random.randn(np.shape(X_test)[0],np.shape(X_test)[1]) * np.mean(X_trainval.to_numpy(),axis=0)*0.03



In [16]:
#Set up network parameters
quantiles=np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
no_nodes=100
est= np.where(quantiles==0.5)[0].item()

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

val_size=0.1
num_models=5 #Set number of ensambles
batch_size=500
nepochs=1000
lr=0.003
noise_ratio = 0.03
early_break=True

In [17]:
#Choose if to save models and metrics, if so set path
save = True
if save:
    test_name = "COT_estimators"
    main_filepath = 'pytorch_models/'+test_name


#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int((len(X_trainval['Cloud_B02'])+len(X_test['Cloud_B02']))*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save:
        filepath=main_filepath+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')

Epoch 311


Batch number: 100%|██████████| 320/320 [00:00<00:00, 384.32it/s]

Training loss [0.45978138] Validation loss [0.4870517]
Epoch 312



Batch number: 100%|██████████| 320/320 [00:00<00:00, 381.90it/s]

Training loss [0.45800662] Validation loss [0.47346357]
Epoch 313


...

Training loss [0.45921057] Validation loss [0.46888846]
Epoch 315



Batch number: 100%|██████████| 320/320 [00:00<00:00, 376.37it/s]

Training loss [0.45519096] Validation loss [0.4726142]
Epoch 316



Batch number: 100%|██████████| 320/320 [00:00<00:00, 383.18it/s]

Training loss [0.45903578] Validation loss [0.4719434]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 216
With validation loss: 0.4663837254047394


In [20]:
#Initialize dataframe for error metrics and array for ensemble predictions
COT_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    MAE=np.mean(np.abs(y_test.to_numpy()-preds[:,:,est]))
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,r2,MAE,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE','Mean_Quantile_Loss'])
    COT_model_metrics=pd.concat([COT_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/num_models

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
MAE=np.mean(np.abs(y_test.to_numpy()-preds_total[:,:,est]))
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,r2,MAE,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE','Mean_Quantile_Loss'])
COT_model_metrics=pd.concat([COT_model_metrics,tmp_metrics])

#Save metrics if we want to
if save:
    COT_model_metrics=COT_model_metrics.reset_index(drop=True)
    COT_model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)
    

/tmp/ipykernel_73203/2016707059.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  COT_model_metrics=pd.concat([COT_model_metrics,tmp_metrics])


In [7]:
#Load in Aleks alg models

#Choose if to save metrics, if so set path
save = True
if save:
    filepath='smhi_models2'

#Set up paths for importing COT est models
COT_model_paths = ['smhi_models2/0/model_it_2000000','smhi_models2/1/model_it_2000000','smhi_models2/2/model_it_2000000','smhi_models2/3/model_it_2000000','smhi_models2/4/model_it_2000000']

#Initialize and load COT estimation models
COT_est_models = [MLP5(12, 1, apply_relu=True) for _ in range(len(COT_model_paths))]
for i,model in enumerate(COT_est_models):
    model.load_state_dict(torch.load(COT_model_paths[i],map_location=device))

#Normalize X_test w.r.t X_train and turn to tensor
X_mu = np.mean(X_trainval.to_numpy(),axis=0)
X_std = np.std(X_trainval.to_numpy(),axis=0)
X_test_norm = (X_test.to_numpy()-X_mu)/X_std
tX_test_norm = torch.Tensor(X_test_norm).to(device)

#Initialize dataframe for error metrics and array for ensemble predictions
COT2_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE'])
preds_total=[]

#Make predictions and find errors
for i,model in enumerate(COT_est_models):
    preds = 50*model(tX_test_norm).cpu().detach().numpy()
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,0])
    r2=r2_score(y_test.to_numpy(),preds[:,0])
    MAE=np.mean(np.abs(y_test.to_numpy()-preds))
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,r2,MAE]],columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE'])
    COT2_model_metrics=pd.concat([COT2_model_metrics,tmp_metrics])

#Now do the same for ensemble predictions
preds_total=preds_total/len(COT_model_paths)

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,0])
r2=r2_score(y_test.to_numpy(),preds_total[:,0])
MAE=np.mean(np.abs(y_test.to_numpy()-preds_total))

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,r2,MAE]],columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE'])
COT2_model_metrics=pd.concat([COT2_model_metrics,tmp_metrics])

if save:
    COT2_model_metrics=COT2_model_metrics.reset_index(drop=True)
    COT2_model_metrics.to_csv(filepath+'/model_metrics.csv',index=False)
    


/tmp/ipykernel_123537/3934616882.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  COT2_model_metrics=pd.concat([COT2_model_metrics,tmp_metrics])


In [31]:
COT_model_metrics

,Ensemble_mean,Ensemble_index,MSE,R2_score,MAE,Mean_Quantile_Loss
0,False,0,18.749951,0.926429,2.097113,7.337143
1,False,1,19.257174,0.924439,2.117094,7.415202
2,False,2,18.775662,0.926328,2.098432,7.330111
3,False,3,18.776029,0.926327,2.112563,7.369339
4,False,4,18.785021,0.926292,2.108568,7.393313
5,True,NaN,17.507435,0.931305,2.032781,7.093999


In [6]:
COT2_model_metrics

,Ensemble_mean,Ensemble_index,MSE,R2_score,MAE
0,False,0,18.310374,0.928154,2.317118
0,False,1,18.760399,0.926388,2.413007
0,False,2,18.938435,0.925690,2.330338
0,False,3,19.081660,0.925128,2.328644
0,False,4,18.662185,0.926774,2.393497
0,True,NaN,17.310384,0.932078,2.255509


Now try to train COT est model with sun zen angle as input aswell

In [4]:
#Find same trainval/test split as Aleksis alg.
#Load training, val and test data for indices
traindata = pd.read_csv('cot_train/data/synthetic-cot-data/train_df.csv', index_col=[0])
valdata = pd.read_csv('cot_train/data/synthetic-cot-data/val_df.csv', index_col=[0])
testdata = pd.read_csv('cot_train/data/synthetic-cot-data/test_df.csv', index_col=[0])

#Merge train and val data
trainvaldata = pd.concat([traindata,valdata])

#Split X's and y's
X_cols = ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
          'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Sun_Zenith_Angle']
y_cols = ['COT']

X_trainval = trainvaldata[X_cols]
X_test = testdata[X_cols]
y_trainval = trainvaldata[y_cols]
y_test = testdata[y_cols]

#Add noise to X_test
np.random.seed(313)
X_test = X_test + np.random.randn(np.shape(X_test)[0],np.shape(X_test)[1]) * np.mean(X_trainval.to_numpy(),axis=0)*0.03


In [5]:
#Set up network parameters
quantiles=np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
no_nodes=100
est= np.where(quantiles==0.5)[0].item()

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

val_size=0.1
num_models=5 #Set number of ensambles
batch_size=500
nepochs=1000
lr=0.003
noise_ratio = 0.03
early_break=True

In [6]:
#Choose if to save models and metrics, if so set path
save = True
if save:
    test_name = "COT_estimators_w_SunZen"
    main_filepath = 'pytorch_models/'+test_name


#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int((len(X_trainval['Cloud_B02'])+len(X_test['Cloud_B02']))*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save:
        filepath=main_filepath+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')

Epoch 351


Batch number: 100%|██████████| 320/320 [00:00<00:00, 357.36it/s]


Training loss [0.37886465] Validation loss [0.38638365]
Epoch 352


Batch number: 100%|██████████| 320/320 [00:00<00:00, 373.00it/s]

Training loss [0.37821856] Validation loss [0.3862812]
Epoch 353


...

Training loss [0.37640718] Validation loss [0.38535565]
Epoch 355



Batch number: 100%|██████████| 320/320 [00:00<00:00, 394.56it/s]

Training loss [0.3762974] Validation loss [0.38419768]
Epoch 356



Batch number: 100%|██████████| 320/320 [00:00<00:00, 396.22it/s]


Training loss [0.37817258] Validation loss [0.38244364]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 256
With validation loss: 0.37934136390686035


In [7]:
#Initialize dataframe for error metrics and array for ensemble predictions
COTwSZ_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    MAE=np.mean(np.abs(y_test.to_numpy()-preds[:,:,est]))
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,r2,MAE,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE','Mean_Quantile_Loss'])
    COTwSZ_model_metrics=pd.concat([COTwSZ_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/num_models

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
MAE=np.mean(np.abs(y_test.to_numpy()-preds_total[:,:,est]))
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,r2,MAE,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','R2_score','MAE','Mean_Quantile_Loss'])
COTwSZ_model_metrics=pd.concat([COTwSZ_model_metrics,tmp_metrics])

#Save metrics if we want to
if save:
    COTwSZ_model_metrics=COTwSZ_model_metrics.reset_index(drop=True)
    COTwSZ_model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)
    

/tmp/ipykernel_151915/3215346210.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  COTwSZ_model_metrics=pd.concat([COTwSZ_model_metrics,tmp_metrics])


In [8]:
COTwSZ_model_metrics

,Ensemble_mean,Ensemble_index,MSE,R2_score,MAE,Mean_Quantile_Loss
0,False,0,12.476870,0.951044,1.762433,6.210402
1,False,1,12.361089,0.951498,1.752743,6.169383
2,False,2,13.077447,0.948687,1.775512,6.250073
3,False,3,12.430350,0.951226,1.745426,6.166187
4,False,4,12.564375,0.950700,1.740261,6.121135
5,True,NaN,11.498256,0.954883,1.682304,5.912684
